In [50]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import linear_model
import nltk
import random
import re
import nltk.corpus
import sklearn
import numpy
import csv
from sklearn import metrics
import sys
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True) #se não ignorar os stopwords, não vai remover eles depois

#inicializando bag de stopwords
stopwords = nltk.corpus.stopwords.words('english')

import os
print(os.listdir("../input"))

['cleandata', 'jigsaw-unintended-bias-in-toxicity-classification']


In [51]:
base = []
csvFile = "../input/cleandata/cleanDataTrain.csv"

baseTrain = []
labels = []
allWords = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            temp1 = row[1]
            allWords.append(temp1)
            baseTrain.append(temp1)
            temp2 = float(row[0])
            labels.append(temp2)
        except IndexError:
            pass

In [52]:
baseTest = []
csvFile = "../input/cleandata/cleanDataTest.csv"
ids = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            # Se for pegar o valor float
            temp2 = row[0]
            temp1 = row[1]
            if(temp2!="id"):
                baseTest.append(temp1)
                ids.append(temp2)
        except IndexError:
            pass

In [53]:
x = baseTrain
y = labels

In [54]:
x_min = []
y_min = []
for u in range(len(y)):
    if(y[u]!=0):
        x_min.append(x[u])
        y_min.append(y[u])

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
#vectorizer_TF = CountVectorizer()
#X_TFIDF = vectorizer_TF.fit_transform(x_min)
#X_TestTFIDF = vectorizer_TF.transform(baseTest)

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_TFIDF = TfidfVectorizer(min_df=0.005)
X_TFIDF = vectorizer_TFIDF.fit_transform(x_min)
X_TestTFIDF = vectorizer_TFIDF.transform(baseTest)

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y_min, test_size=0.10, random_state=42)

In [58]:
X_train[0].shape

(1, 1052)

In [59]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Input, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

#X_train = X_train[:200000]
#y_train = y_train[:200000]

input_layer = Input(shape=X_train[0].shape)
conv1 = Conv1D(filters=32,
               kernel_size=8,
               strides=1,
               activation='relu',
               padding='same')(input_layer)
lstm1 = LSTM(32, return_sequences=True)(conv1)
conv2 = Conv1D(filters=32,
               kernel_size=8,
               strides=1,
               activation='relu',
               padding='same')(lstm1)
lstm2 = LSTM(32, return_sequences=True)(conv2)
conv3 = Conv1D(filters=32,
               kernel_size=8,
               strides=1,
               activation='relu',
               padding='same')(lstm2)
lstm3 = LSTM(32, return_sequences=True)(conv3)
output_layer = Dense(1, activation='sigmoid')(lstm3)
model = Model(inputs=input_layer, outputs=output_layer)

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Classificação com probabilidade
y_train_class = []
for t in range(len(y_train)):
    if(y_train[t]<0.5):
        y_train_class.append(0)
    else:
        y_train_class.append(1)

#X_test = (X_test.toarray()).reshape(1,X_test.shape[0], X_test.shape[1])
y_train_class = numpy.asarray(y_train_class)
y_train_class.reshape(1,len(y_train_class),1)
X_train = (X_train.toarray()).reshape(X_train.shape[0],1, X_train.shape[1])

print(X_train.shape)
print(y_train_class.shape)
y_train_class = numpy.expand_dims(y_train_class, axis=1)
print(y_train_class.shape)
y_train_class = numpy.expand_dims(y_train_class, axis=2)
print(y_train_class.shape)

model.fit(X_train, y_train_class, epochs=100, batch_size=1024)
scores = model.evaluate(X_train, y_train_class)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1, 1052)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1, 32)             269344    
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 32)             8320      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1, 32)             8224      
_________________________________________________________________
lstm_6 (LSTM)                (None, 1, 32)             8320      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1, 32)             8224      
_________________________________________________________________
lstm_7 (LSTM)                (None, 1, 32)             8320      
__________

KeyboardInterrupt: 

In [41]:
from sklearn import metrics
#y_pred = clf.predict(X_test) 
X_test = (X_test.toarray()).reshape(X_test.shape[0], 1, X_test.shape[1])
y_pred = model.predict(X_test) 
y_pred_prob = y_pred
#print(y_pred_prob)
#print(X_test.shape)

y_pred_class = []
y_test_class = []
y_pred_prob_tox = []

for t in range(len(y_pred)):
    if(y_pred[t][0]<0.5):
        y_pred_class.append(0)
    else:
        y_pred_class.append(1)

for t in range(len(y_test)):
    if(y_test[t]<0.5):
        y_test_class.append(0)
    else:
        y_test_class.append(1)
#    y_pred_prob_tox.append(y_pred_prob[t][0])

#print(y_test_class[0], y_pred_class[0])
#print(y_test.shape)
#print(y_pred.shape)
y_pred = y_pred.reshape(y_pred.shape[0], y_pred.shape[1])

mse = metrics.mean_squared_error(y_test, y_pred)
auc = metrics.roc_auc_score(y_test_class, y_pred_class)
acc = metrics.accuracy_score(y_test_class, y_pred_class)
prec = metrics.precision_score(y_test_class, y_pred_class)
rec = metrics.recall_score(y_test_class, y_pred_class)

print(auc)
print(acc)
print(prec)
print(rec)
print(mse)

with open('results_validation.csv', mode='w') as employee_file:
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Logistic Regression"])
    writer.writerow(["mse: ", mse])
    writer.writerow(["auc: ", auc])
    writer.writerow(["accuracy: ", acc])
    writer.writerow(["precision: ", prec])
    writer.writerow(["recall: ", rec])

0.700945261184793
0.8136861009794302
0.7381191993738119
0.4608993157380254
0.04751784805741493


In [49]:
X_TestTFIDF = (X_TestTFIDF.toarray()).reshape(X_TestTFIDF.shape[0], 1, X_TestTFIDF.shape[1])
y_pred_test = model.predict(X_TestTFIDF)
with open('submission.csv', mode='w') as employee_file:
    
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "prediction"])
    for id in range(len(ids)):
        print(ids[id])
        print(y_pred_test[id][0][0])
        writer.writerow([ids[id], y_pred_test[id][0][0]])

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'